In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from  sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble  import GradientBoostingClassifier 
from lightgbm import LGBMClassifier 
from  sklearn.model_selection import  train_test_split 

In [2]:
data = pd.read_csv('./preprocesss_train_test_all.csv')

In [3]:
train_size = len(data) -2968
train_data   = data.iloc[ 0:train_size , :]
test_data = data.iloc[train_size: ,: ] 
train_data = train_data.drop( ['id'] ,axis =1   )

test_sub= test_data.loc[ :, ['id']]
test_data  = test_data.drop( ['id' , 'happiness' ] ,axis =1   ) 

In [4]:
y_train ,x_train =  train_data['happiness'] ,train_data.drop( 'happiness' ,axis =1 )
x_test = test_data

In [5]:
y_train =y_train.astype('int64') -1 

In [6]:
X_train,X_valid, Y,Y_valid = train_test_split(x_train , 
                                              y_train ,
                                              test_size = 0.2 ,
                                              random_state  = 17  )
print (X_train.shape , X_valid.shape , Y.shape , Y_valid.shape)

(6400, 133) (1600, 133) (6400,) (1600,)


In [12]:
##### lgb
from  sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

param = {'boosting_type': 'gbdt',
         'num_leaves': 20,
         'min_data_in_leaf': 20, 
         'objective':'regression',
         'max_depth':6,
         'learning_rate': 0.01,
         "min_child_samples": 30,

         "feature_fraction": 0.8,
         "bagging_freq": 1,
         "bagging_fraction": 0.8 ,
         "bagging_seed": 11,
         "metric": 'mse',
         "lambda_l1": 0.1,
         "verbosity": -1}
folds = KFold(n_splits=5, shuffle=True, random_state=2018)
oof_lgb = np.zeros(len(X_train))
predictions_lgb = np.zeros(len(x_test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, Y)):
    print("fold n°{}".format(fold_+1))
    trn_data = lgb.Dataset(X_train.iloc[trn_idx],  Y.iloc[trn_idx])
    val_data = lgb.Dataset(X_train.iloc[val_idx],  Y.iloc[val_idx])

    num_round = 10000
    clf = lgb.train(param, trn_data,
                    num_round, 
                    valid_sets = [trn_data, val_data], 
                    verbose_eval=200, 
                    early_stopping_rounds = 100)
    oof_lgb[val_idx] = clf.predict(X_train.iloc[val_idx], 
                                   num_iteration=clf.best_iteration)

    predictions_lgb += clf.predict(x_test, num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.8f}".format(mean_squared_error(oof_lgb, Y)))

fold n°1
Training until validation scores don't improve for 100 rounds
[200]	training's l2: 0.29796	valid_1's l2: 0.339481
[400]	training's l2: 0.254405	valid_1's l2: 0.327559
[600]	training's l2: 0.227084	valid_1's l2: 0.324552
[800]	training's l2: 0.204546	valid_1's l2: 0.323406
Early stopping, best iteration is:
[736]	training's l2: 0.21125	valid_1's l2: 0.32323
fold n°2
Training until validation scores don't improve for 100 rounds
[200]	training's l2: 0.291468	valid_1's l2: 0.35907
[400]	training's l2: 0.247803	valid_1's l2: 0.352483
Early stopping, best iteration is:
[450]	training's l2: 0.239672	valid_1's l2: 0.352289
fold n°3
Training until validation scores don't improve for 100 rounds
[200]	training's l2: 0.291424	valid_1's l2: 0.365961
[400]	training's l2: 0.249339	valid_1's l2: 0.354856
[600]	training's l2: 0.221037	valid_1's l2: 0.353298
Early stopping, best iteration is:
[629]	training's l2: 0.217623	valid_1's l2: 0.353064
fold n°4
Training until validation scores don't im

In [14]:
##### xgb
import xgboost as xgb
#自定义评价函数
def msefunc(preds, xgbtrain):
    label = xgbtrain.get_label()
    score = mean_squared_error(label,preds)
    return 'myFeval',score

xgb_params = {"booster":'gbtree','eta': 0.005, 'max_depth': 5, 'subsample': 0.7, 
              'colsample_bytree': 0.8, 'objective': 'reg:linear', 'eval_metric': 'rmse', 'silent': True, 'nthread': 8}
folds = KFold(n_splits=5, shuffle=True, random_state=2018)
oof_xgb = np.zeros(len(X_train))
predictions_xgb = np.zeros(len(x_test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, Y)):
    print("fold n°{}".format(fold_+1))
    trn_data = xgb.DMatrix(X_train.iloc[trn_idx ,:], Y.iloc[trn_idx])
    val_data = xgb.DMatrix(X_train.iloc[val_idx,: ], Y.iloc[val_idx])
    
    watchlist = [(trn_data, 'train'), (val_data, 'valid_data')]
    clf = xgb.train(dtrain=trn_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200, verbose_eval=100, params=xgb_params,feval = msefunc)
    oof_xgb[val_idx] = clf.predict(xgb.DMatrix(X_train.iloc[val_idx ,:]), ntree_limit=clf.best_ntree_limit)
    predictions_xgb += clf.predict(xgb.DMatrix(x_test), ntree_limit=clf.best_ntree_limit) / folds.n_splits
    
print("CV score: {:<8.8f}".format(mean_squared_error(oof_xgb, Y)))

fold n°1
[0]	train-rmse:2.47777	valid_data-rmse:2.50704	train-myFeval:6.13937	valid_data-myFeval:6.28524
Multiple eval metrics have been passed: 'valid_data-myFeval' will be used for early stopping.

Will train until valid_data-myFeval hasn't improved in 200 rounds.
[100]	train-rmse:1.57189	valid_data-rmse:1.60494	train-myFeval:2.47083	valid_data-myFeval:2.57584
[200]	train-rmse:1.05120	valid_data-rmse:1.09122	train-myFeval:1.10501	valid_data-myFeval:1.19076
[300]	train-rmse:0.76799	valid_data-rmse:0.81751	train-myFeval:0.58980	valid_data-myFeval:0.66832
[400]	train-rmse:0.62386	valid_data-rmse:0.68400	train-myFeval:0.38920	valid_data-myFeval:0.46786
[500]	train-rmse:0.55373	valid_data-rmse:0.62376	train-myFeval:0.30662	valid_data-myFeval:0.38908
[600]	train-rmse:0.51881	valid_data-rmse:0.59768	train-myFeval:0.26916	valid_data-myFeval:0.35722
[700]	train-rmse:0.49928	valid_data-rmse:0.58591	train-myFeval:0.24928	valid_data-myFeval:0.34329
[800]	train-rmse:0.48610	valid_data-rmse:0.5801

fold n°5
[0]	train-rmse:2.48298	valid_data-rmse:2.48611	train-myFeval:6.16520	valid_data-myFeval:6.18077
Multiple eval metrics have been passed: 'valid_data-myFeval' will be used for early stopping.

Will train until valid_data-myFeval hasn't improved in 200 rounds.
[100]	train-rmse:1.57431	valid_data-rmse:1.58490	train-myFeval:2.47846	valid_data-myFeval:2.51190
[200]	train-rmse:1.05128	valid_data-rmse:1.07444	train-myFeval:1.10519	valid_data-myFeval:1.15442
[300]	train-rmse:0.76677	valid_data-rmse:0.80666	train-myFeval:0.58794	valid_data-myFeval:0.65070
[400]	train-rmse:0.62218	valid_data-rmse:0.67989	train-myFeval:0.38711	valid_data-myFeval:0.46226
[500]	train-rmse:0.55226	valid_data-rmse:0.62495	train-myFeval:0.30500	valid_data-myFeval:0.39056
[600]	train-rmse:0.51726	valid_data-rmse:0.60260	train-myFeval:0.26756	valid_data-myFeval:0.36312
[700]	train-rmse:0.49773	valid_data-rmse:0.59301	train-myFeval:0.24774	valid_data-myFeval:0.35166
[800]	train-rmse:0.48456	valid_data-rmse:0.5888

In [15]:
from sklearn import linear_model
from  sklearn.model_selection import RepeatedKFold

# 将lgb和xgb和ctb的结果进行stacking
train_stack =  pd.concat( [ pd.Series(oof_lgb ) ,pd.Series( oof_xgb ) ] ,axis =1 ) 
test_stack = pd.concat( [ pd.Series(predictions_lgb) ,
                         pd.Series( predictions_xgb )  ] ,axis =1 ) 


folds_stack = RepeatedKFold(n_splits=5, n_repeats=2, random_state=2018)
oof_stack = np.zeros(train_stack.shape[0])
predictions = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds_stack.split(train_stack,Y)):
    trn_data, trn_y = train_stack.iloc[ trn_idx,: ], Y.iloc[trn_idx ]
    val_data, val_y = train_stack.iloc[ val_idx ,:  ], Y.iloc[val_idx]

    clf_stack =linear_model.LinearRegression()
    clf_stack.fit(trn_data, trn_y)
    
    oof_stack[val_idx] = clf_stack.predict(val_data)
    predictions += clf_stack.predict(test_stack) / 10
    
print("CV score: {:<8.8f}".format(mean_squared_error(oof_stack, Y)))


CV score: 0.33644929


In [19]:
result=list(predictions)
result=list(map(lambda x: x + 1, result))
test_sub["happiness"]=result
test_sub.to_csv("submit_02.csv", index=False)

In [20]:
test_sub

,id,happiness
8000,8001,3.899866
8001,8002,3.477392
8002,8003,3.439279
8003,8004,4.174374
8004,8005,3.671427
...,...,...
10963,10964,4.312923
10964,10965,3.689481
10965,10966,4.044440
10966,10967,3.913298


In [21]:
len(data)

10968